In [1]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

import tensorflow as tf
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# GRADED FUNCTION: split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets
  
  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training
    
  Returns:
    None
  """

  ### START CODE HERE
  file_list = []
  for i in os.listdir(SOURCE_DIR):
    if os.path.getsize(os.path.join(SOURCE_DIR, i)) == 0:
      print(i, 'is zero length, so ignoring.')
      continue

    file_list.append(i)

  random_sample = random.sample(file_list, len(file_list))
  indices = int(split_size*len(random_sample))
  for i in random_sample[:indices]:
    copyfile(os.path.join(SOURCE_DIR, i), os.path.join(TRAINING_DIR, i))
  for i in random_sample[indices:]:
    copyfile(os.path.join(SOURCE_DIR, i), os.path.join(VALIDATION_DIR, i))
  ### END CODE HERE

In [3]:
# Define root directory
root_dir = './malaria/'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# GRADED FUNCTION: create_train_val_dirs
def create_train_val_dirs(root_path):
  """
  Creates directories for the train and test sets
  
  Args:
    root_path (string) - the base directory path to create subdirectories from
  
  Returns:
    None
  """  
  ### START CODE HERE

  # HINT:
  # Use os.makedirs to create your directories with intermediate subdirectories
  # Don't hardcode the paths. Use os.path.join to append the new directories to the root_path parameter
  os.makedirs(os.path.join(root_path, "training/Parasitized"))
  os.makedirs(os.path.join(root_path, "training/Uninfected"))
  os.makedirs(os.path.join(root_path, "validation/Uninfected"))
  os.makedirs(os.path.join(root_path, "validation/Parasitized"))

  ### END CODE HERE

  
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [4]:
parasitised = "./cell_images/Parasitized/"
normal = "./cell_images/Uninfected/"

training_dir = "./malaria/training/"
validation_dir = "./malaria/validation"


In [5]:
# GRADED FUNCTION: split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets
  
  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training
    
  Returns:
    None
  """

  ### START CODE HERE
  file_list = []
  for i in os.listdir(SOURCE_DIR):
    if os.path.getsize(os.path.join(SOURCE_DIR, i)) == 0:
      print(i, 'is zero length, so ignoring.')
      continue

    file_list.append(i)

  random_sample = random.sample(file_list, len(file_list))
  indices = int(split_size*len(random_sample))
  for i in random_sample[:indices]:
    copyfile(os.path.join(SOURCE_DIR, i), os.path.join(TRAINING_DIR, i))
  for i in random_sample[indices:]:
    copyfile(os.path.join(SOURCE_DIR, i), os.path.join(VALIDATION_DIR, i))
  ### END CODE HERE


In [6]:
training_dir_parasitised = os.path.join(training_dir, "Parasitized")
training_dir_normal = os.path.join(training_dir, "Uninfected")
validation_dir_parasitised = os.path.join(validation_dir, "Parasitized")
validation_dir_normal = os.path.join(validation_dir, "Uninfected")

In [7]:
# Empty directories in case you run this cell multiple times
if len(os.listdir(training_dir_normal)) > 0:
  for file in os.scandir(training_dir_normal):
    os.remove(file.path)
if len(os.listdir(training_dir_parasitised)) > 0:
  for file in os.scandir(training_dir_parasitised):
    os.remove(file.path)
if len(os.listdir(validation_dir_normal)) > 0:
  for file in os.scandir(validation_dir_normal):
    os.remove(file.path)
if len(os.listdir(validation_dir_parasitised)) > 0:
  for file in os.scandir(validation_dir_parasitised):
    os.remove(file.path)

# Define proportion of images used for training
split_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(normal, training_dir_normal, validation_dir_normal, split_size)
split_data(parasitised, training_dir_parasitised, validation_dir_parasitised, split_size)


In [8]:
# GRADED FUNCTION: train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators
  
  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images
    
  Returns:
    train_generator, validation_generator - tuple containing the generators
  """
  ### START CODE HERE

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator(rescale = 1.0/255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=1024,
                                                      class_mode='binary',
                                                      target_size=(128, 128))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale = 1.0/255.)

  # Pass in the appropiate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=512,
                                                                class_mode='binary',
                                                                target_size=(128, 128))
  ### END CODE HERE
  return train_generator, validation_generator


In [9]:
train_generator, validation_generator = train_val_generators(training_dir, validation_dir)

Found 24802 images belonging to 2 classes.
Found 2756 images belonging to 2 classes.


In [10]:
# GRADED FUNCTION: create_model
def create_model():
  # DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
  # USE AT LEAST 3 CONVOLUTION LAYERS

  ### START CODE HERE

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', padding = 'same', input_shape = (128,128,3)),
      tf.keras.layers.MaxPool2D((3,3)),

      tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same'),
      tf.keras.layers.MaxPool2D((3,3)),

      tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same'),
      tf.keras.layers.MaxPool2D((3,3)),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(256, activation = 'relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(512, activation = 'relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])

  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss='binary_crossentropy',
                metrics=['accuracy']) 
    
  ### END CODE HERE

  return model

# Get the untrained model
model = create_model()

# Train the model
# Note that this may take some time.
with tf.device("GPU:0"):
    history = model.fit(train_generator,
                        epochs=25,
                        verbose=1,
                        validation_data=validation_generator)

Epoch 1/25
25/25 [==============================] - 23s 642ms/step - loss: 1.2707 - accuracy: 0.5046 - val_loss: 0.6921 - val_accuracy: 0.5000
Epoch 2/25
25/25 [==============================] - 15s 614ms/step - loss: 0.6865 - accuracy: 0.5479 - val_loss: 0.6675 - val_accuracy: 0.6114
Epoch 3/25
25/25 [==============================] - 15s 612ms/step - loss: 0.6766 - accuracy: 0.5738 - val_loss: 0.6816 - val_accuracy: 0.6277
Epoch 4/25
25/25 [==============================] - 16s 627ms/step - loss: 0.6526 - accuracy: 0.6562 - val_loss: 0.6021 - val_accuracy: 0.7148
Epoch 5/25
25/25 [==============================] - 16s 630ms/step - loss: 0.5615 - accuracy: 0.7184 - val_loss: 0.5288 - val_accuracy: 0.7253
Epoch 6/25
25/25 [==============================] - 15s 608ms/step - loss: 0.4603 - accuracy: 0.7823 - val_loss: 0.4152 - val_accuracy: 0.8022
Epoch 7/25
25/25 [==============================] - 15s 610ms/step - loss: 0.4335 - accuracy: 0.7987 - val_loss: 0.3488 - val_accuracy: 0.8451

In [12]:
#model.save("great_model_95pct")
#model.save_weights("95_pct_weights")

INFO:tensorflow:Assets written to: great_model_95pct\assets


INFO:tensorflow:Assets written to: great_model_95pct\assets
